In [ ]:
from github import Github, GithubException, BadCredentialsException
import json
from datetime import datetime
import time
import os

In [ ]:
access_token = os.environ['GITHUB_API_TOKEN']
user = Github(access_token)

In [ ]:
FILE_NAME = './data/repos.json'

In [ ]:
def check_rate_limit():
    rate_limit = user.get_rate_limit()
    remaining = rate_limit.search.remaining

    if remaining > 0:
        return

    reset_time = rate_limit.search.reset
    now = datetime.utcnow()

    if now < reset_time:
        sleep_time = (reset_time - now).total_seconds()
        print('Waiting %s seconds for rate limit reset.' % sleep_time)
        time.sleep(sleep_time)

In [ ]:
def repo_details(repo):
    check_rate_limit()
    
    print('Getting details for repo %s …' % repo.full_name)
    
    return {
        'name': repo.full_name,
        'language': repo.language,
        'created_at': repo.created_at.isoformat(),
        'forks': repo.forks,
        'size': repo.size,
        'watchers': repo.watchers,
        'url': repo.url,
    }

In [ ]:
def search(query, sort, order):
    repos = user.search_repositories(query, sort, order, created = '>=2018-01-01')
    return [repo_details(repo) for repo in repos]

In [ ]:
try:
    start = time.time()
    repos = search('gameoflife', 'stars', 'desc')
    print('Searching %d repos took %f seconds.' % (len(repos), round(time.time() - start, 4)))
    with open(FILE_NAME, 'w+', encoding = 'utf-8') as f:
        json.dump(repos, f, sort_keys = True, indent = 2)
        print('Results written into %s !' % FILE_NAME)
except BadCredentialsException as e:
    print(e)
except GithubException as e:
    print(e.message)